In [1]:
# System
import os

# Data processing
import numpy as np
import pandas as pd
import math as m
import datetime as dt
import random

# Results presentation
from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt

# NN related stuff
import torch
# import torch.nn as nn
# import torch.nn.functional as F
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve



%matplotlib inline

In [2]:
SAVE_INFO = False

In [3]:
if (SAVE_INFO == True):
    DATA_DIR = 'Data/'
    EXPERIMENT_DIR = DATA_DIR + 'Experiment_conv/'
    os.makedirs(EXPERIMENT_DIR)

In [4]:
LEFT_BORDER = 0
RIGHT_BORDER = 2000
DOWN_BORDER = 0
UP_BORDER = 2500

N_CELLS_HOR = 200
N_CELLS_VER = 250

R_CIRCLE = 50

In [5]:
celled_data = torch.load("Data/celled_data_" +
                         str(N_CELLS_HOR) + "x" +
                         str(N_CELLS_VER))

celled_data_RTL = torch.load ("Data/RTL_features")

In [6]:
print (celled_data    .shape)
print (celled_data_RTL.shape)

torch.Size([9591, 1, 200, 250])
torch.Size([9591, 9, 200, 250])


In [7]:
def show_one_day_image (tensor, day):
    plt.imshow (tensor[day].squeeze(0), cmap=plt.cm.afmhot_r)
    plt.colorbar()

def show_one_day_image_RTL (tensor):
    plt.imshow (tensor, cmap=plt.cm.afmhot_r)
    plt.colorbar()
    plt.show
    
def show_one_day_quakes (tensor, day):
    state = tensor[day].squeeze(0)
    print (state.shape)
    X = []
    Y = []
    M = []
    for i in range(state.shape[0]):
        for j in range(state.shape[1]):
            if (state[i][j] != 0):
                X.append(i)
                Y.append(j)
                M.append(state[i][j].item())
    print (X)
    print (Y)
    print (M)
    
    fig = plt.figure(figsize=(8, 10))
    axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
    plt.axis([0, state.shape[0], 0, state.shape[1]])
    axes.scatter(X, Y, s=500, c=M, marker='.', cmap=plt.cm.Reds)
#     plt.colorbar()
    

#### Распечатаем сколько землетрясений и с какой амплитудой случалось

# Обучение

In [8]:
def my_accuracy (input, target, threshold):
    true = ((input>threshold) == target)
    sum = torch.sum (true.float())
    return sum/input.shape[0]/input.shape[1]/input.shape[2]/input.shape[3]

def my_precision (input, target, threshold):
    TP = torch.sum (((input>threshold) * target      ).float())
    FP = torch.sum (((input>threshold) * (1 - target)).float())
    return TP / (TP + FP)

def my_recall (input, target, threshold):
    TP = torch.sum ((     (input>threshold)  * target).float())
    FN = torch.sum (((1 - (input>threshold)) * target).float())
    return TP / (TP + FN)

def my_precision_recall (input, target, threshold):
    TP = torch.sum ((     (input>threshold)  * target      ).float())
    FP = torch.sum ((     (input>threshold)  * (1 - target)).float())
    FN = torch.sum (((1 - (input>threshold)) * target      ).float())
#     print ('TP = ', TP.item(), 'FP = ', FP.item(), 'FN = ', FN.item(), 'N = ', input.shape[0])
    return TP / (TP + FP), TP / (TP + FN)

### Создадим датасет
#### (Может не влезть в оперативку (надо ~ 12Gb))

In [9]:
OBSERVED_DAYS = 20     # ~2 months
DAYS_TO_PREDICT_AFTER  = 10
DAYS_TO_PREDICT_BEFORE = 50
TESTING_DAYS = 1000

HEAVY_QUAKE_THRES = 3.5

In [10]:
print (celled_data.shape)

torch.Size([9591, 1, 200, 250])


In [11]:
def create_X_and_Y_pos():
    Y_more = np.array(celled_data > HEAVY_QUAKE_THRES)
    X_train = np.zeros([0, celled_data_RTL.shape[1]], dtype=float)
    Y_train = np.zeros([0])
    for t in tqdm (range (OBSERVED_DAYS, celled_data.shape[0]-
                                         DAYS_TO_PREDICT_BEFORE)):
#         print (X_train.shape)
#         print (Y_train.shape)
        for i in range (celled_data.shape[2]):
            for j in range (celled_data.shape[3]):
                X = np.array(celled_data_RTL[t, :, i, j])
                Y = np.sum(Y_more[(t + DAYS_TO_PREDICT_AFTER):
                                  (t + DAYS_TO_PREDICT_BEFORE), 0, i, j]) > 0
                if (Y > 0):
#                     print ("Adding", t, i, j)
                    X_train = np.concatenate((X_train, np.expand_dims(X, axis=0)), axis=0)
                    Y_train = np.concatenate((Y_train, np.expand_dims(Y, axis=0)), axis=0)
    return X_train, Y_train

def create_X_and_Y_neg():
    Y_more = np.array(celled_data > HEAVY_QUAKE_THRES)
    X_train = np.zeros([0, celled_data_RTL.shape[1]], dtype=float)
    Y_train = np.zeros([0])
    for t in tqdm (range (OBSERVED_DAYS, celled_data.shape[0]-
                                         DAYS_TO_PREDICT_BEFORE)):
#         print (X_train.shape)
#         print (Y_train.shape)
        for i in range (celled_data.shape[2]):
            for j in range (celled_data.shape[3]):
                X = np.array(celled_data_RTL[t, :, i, j])
                Y = np.sum(Y_more[(t + DAYS_TO_PREDICT_AFTER):
                                  (t + DAYS_TO_PREDICT_BEFORE), 0, i, j]) > 0
                if (Y == 0 and random.randint(1, 500) == 500):
                    print ("Adding", t, i, j, Y)
                    X_train = np.concatenate((X_train, np.expand_dims(X, axis=0)), axis=0)
                    Y_train = np.concatenate((Y_train, np.expand_dims(Y, axis=0)), axis=0)
    return X_train, Y_train

In [ ]:
X, Y = create_X_and_Y_neg()

Adding 20 2 235 False
Adding 20 5 125 False
Adding 20 5 242 False
Adding 20 7 130 False
Adding 20 9 195 False
Adding 20 14 166 False
Adding 20 14 224 False
Adding 20 15 214 False
Adding 20 19 173 False
Adding 20 23 249 False
Adding 20 25 156 False
Adding 20 25 164 False
Adding 20 32 124 False
Adding 20 33 206 False
Adding 20 34 243 False
Adding 20 35 19 False
Adding 20 37 132 False
Adding 20 38 204 False
Adding 20 39 142 False
Adding 20 40 128 False
Adding 20 41 141 False
Adding 20 42 98 False
Adding 20 42 166 False
Adding 20 44 64 False
Adding 20 45 36 False
Adding 20 45 107 False
Adding 20 48 110 False
Adding 20 50 76 False
Adding 20 52 114 False
Adding 20 52 173 False
Adding 20 54 138 False
Adding 20 55 175 False
Adding 20 55 220 False
Adding 20 56 12 False
Adding 20 59 135 False
Adding 20 61 209 False
Adding 20 68 127 False
Adding 20 70 57 False
Adding 20 71 99 False
Adding 20 71 174 False
Adding 20 73 201 False
Adding 20 73 235 False
Adding 20 74 64 False
Adding 20 83 171 False
Ad

Adding 23 115 115 False
Adding 23 115 127 False
Adding 23 116 56 False
Adding 23 118 61 False
Adding 23 127 162 False
Adding 23 128 218 False
Adding 23 129 15 False
Adding 23 129 139 False
Adding 23 129 200 False
Adding 23 131 127 False
Adding 23 133 41 False
Adding 23 136 33 False
Adding 23 136 60 False
Adding 23 143 43 False
Adding 23 147 18 False
Adding 23 148 96 False
Adding 23 149 93 False
Adding 23 150 46 False
Adding 23 150 157 False
Adding 23 151 248 False
Adding 23 153 53 False
Adding 23 154 195 False
Adding 23 160 249 False
Adding 23 162 156 False
Adding 23 163 41 False
Adding 23 170 127 False
Adding 23 171 0 False
Adding 23 178 87 False
Adding 23 180 61 False
Adding 23 180 159 False
Adding 23 180 235 False
Adding 23 181 193 False
Adding 23 182 120 False
Adding 23 182 181 False
Adding 23 185 74 False
Adding 23 188 23 False
Adding 23 190 78 False
Adding 23 190 126 False
Adding 23 193 105 False
Adding 23 193 201 False
Adding 23 194 56 False
Adding 23 196 187 False
Adding 23 198

In [ ]:
print (X.shape)
print (Y.shape)

In [ ]:
np.savetxt("Data/boosting_RTL/X_neg", X)
np.savetxt("Data/boosting_RTL/Y_neg", Y)

In [15]:
# X_train = np.loadtxt("Data/boosting_RTL/X_pos")
# Y_train = np.loadtxt("Data/boosting_RTL/Y_pos")
# print (X_train.shape)
# print (Y_train.shape)

(951868, 9)
(951868,)


In [ ]:
# X_train, Y_train = create_X_and_Y_train()

In [ ]:
# np.savetxt("Data/train_boosting/X", X_train)
# np.savetxt("Data/train_boosting/Y", Y_train)

In [ ]:
# X_test , Y_test  = create_X_and_Y_test ()

In [ ]:
# np.savetxt("Data/test_boosting/X" , X_test)
# np.savetxt("Data/test_boosting/Y" , Y_test)

In [ ]:
# def create_X_and_Y_train():
#     X_train = np.zeros([0, OBSERVED_DAYS], dtype=float)
#     Y_train = np.zeros([0])
#     for t in tqdm (range (OBSERVED_DAYS, celled_data.shape[0] -
#                                          TESTING_DAYS -
#                                          DAYS_TO_PREDICT_BEFORE)):
#         for i in range (celled_data.shape[2]):
#             for j in range (celled_data.shape[3]):
#                 X = celled_data[(t - OBSERVED_DAYS):t, 0, i, j]
#                 Y = np.sum((celled_data[(t + DAYS_TO_PREDICT_AFTER):
#                                         (t + DAYS_TO_PREDICT_BEFORE), 0, i, j] >
#                             HEAVY_QUAKE_THRES)) > 0
#                 if (np.sum(X) > 0):
#                     X_train = np.concatenate((X_train, np.expand_dims(X, axis=0)), axis=0)
#                     Y_train = np.concatenate((Y_train, np.expand_dims(Y, axis=0)), axis=0)

#     #             print (X_train.shape)
#     #             print (Y_train.shape)
#     return X_train, Y_train

# def create_X_and_Y_test():
#     X_test = np.zeros([0, OBSERVED_DAYS], dtype=float)
#     Y_test = np.zeros([0])
#     for t in tqdm (range (celled_data.shape[0] -
#                           TESTING_DAYS +
#                           OBSERVED_DAYS,
#                           celled_data.shape[0] -
#                           DAYS_TO_PREDICT_BEFORE)):
#         for i in range (celled_data.shape[2]):
#             for j in range (celled_data.shape[3]):
#                 X = celled_data[(t - OBSERVED_DAYS):t, 0, i, j]
#                 Y = np.sum((celled_data[(t + DAYS_TO_PREDICT_AFTER):
#                                         (t + DAYS_TO_PREDICT_BEFORE), 0, i, j] >
#                             HEAVY_QUAKE_THRES)) > 0
#                 if (np.sum(X) > 0):
#                     X_test = np.concatenate((X_test, np.expand_dims(X, axis=0)), axis=0)
#                     Y_test = np.concatenate((Y_test, np.expand_dims(Y, axis=0)), axis=0)

#     #             print (X_test.shape)
#     #             print (Y_test.shape)
#     return X_test, Y_test

In [ ]:
X_train = np.loadtxt("Data/train_boosting/X")
Y_train = np.loadtxt("Data/train_boosting/Y")
print (X_train.shape)
print (Y_train.shape)

In [ ]:
X_test  = np.loadtxt("Data/test_boosting/X")
Y_test  = np.loadtxt("Data/test_boosting/Y")
print (X_test .shape)
print (Y_test .shape)

In [ ]:
# X_train = np.zeros([(celled_data.shape[0] -
#                      OBSERVED_DAYS -
#                      TESTING_DAYS -
#                      DAYS_TO_PREDICT_BEFORE) *
#                     N_CELLS_HOR *
#                     N_CELLS_VER,
#                     OBSERVED_DAYS], 
#                    dtype=float)
# Y_train = np.zeros([(celled_data.shape[0] -
#                      OBSERVED_DAYS -
#                      TESTING_DAYS -
#                      DAYS_TO_PREDICT_BEFORE) *
#                     N_CELLS_HOR *
#                     N_CELLS_VER,
#                     OBSERVED_DAYS])

# for t in tqdm (range (OBSERVED_DAYS, celled_data.shape[0] -
#                                      TESTING_DAYS -
#                                      DAYS_TO_PREDICT_BEFORE)):
#     for i in range (celled_data.shape[2]):
#         for j in range (celled_data.shape[3]):
#             X_train[t * N_CELLS_HOR * N_CELLS_VER + i * N_CELLS_VER + j] = \
#             celled_data[(t - OBSERVED_DAYS):t, 0, i, j]
            
#             Y_train[t * N_CELLS_HOR * N_CELLS_VER + i * N_CELLS_VER + j] = \
#             np.sum(celled_data[(t + DAYS_TO_PREDICT_AFTER):
#                                (t + DAYS_TO_PREDICT_BEFORE), 0, i, j] >
#                    HEAVY_QUAKE_THRES) > 0
# #             X_train = np.concatenate((X_train, np.expand_dims(X, axis=0)), axis=0)
# #             Y_train = np.concatenate((Y_train, np.expand_dims(Y, axis=0)), axis=0)
            
# #             print (X_train.shape)
# #             print (Y_train.shape)

### Создадим саму сеть

In [ ]:
# dtrain = xgb.DMatrix (X_train, label=Y_train)
# dtest  = xgb.DMatrix (X_test)

In [ ]:
boost_clf = xgb.XGBClassifier(objective='binary:logistic', n_jobs=4)
boost_clf.fit(X_train, Y_train)

In [ ]:
prediction = boost_clf.predict(X_test)

ROC_AUC_score = roc_auc_score(Y_test, prediction)
Avg_precision = average_precision_score(Y_test, prediction)

print ("ROC_AUC = ", ROC_AUC_score, "Avg_precision = ", Avg_precision)

### GridSearch

In [ ]:
boost_clf = xgb.XGBClassifier(objective='binary:logistic', n_jobs=16)

param_grid = {'n_jobs'      :[16],
              'objective'   :['binary:logistic'],
              'max_depth'   :[1, 3, 5, 11],
              'n_estimators':[2, 10, 20],
              'booster'     :['gbtree', 'gblinear']
             } # might be useful to try to change some other params
grid_cv = GridSearchCV (boost_clf, param_grid, scoring=make_scorer(average_precision_score), cv=5)

grid_cv.fit(X_train, Y_train)

In [ ]:
print ('Best params:')
print (grid_cv.best_params_)

best_boost_clf = xgb.XGBClassifier(n_jobs      =grid_cv.best_params_['n_jobs'],
                                   objective   =grid_cv.best_params_['objective'],
                                   max_depth   =grid_cv.best_params_['max_depth'],
                                   n_estimators=grid_cv.best_params_['n_estimators'],
                                   booster     =grid_cv.best_params_['booster'])

best_boost_clf.fit (X_train, Y_train)

In [ ]:
prediction = best_boost_clf.predict(X_test)

ROC_AUC_score = roc_auc_score(Y_test, prediction)
Avg_precision = average_precision_score(Y_test, prediction)

print ("ROC_AUC = ", ROC_AUC_score, "Avg_precision = ", Avg_precision)

In [ ]:
for i in range (4, 21, 2):
    boost_clf = xgb.XGBClassifier(n_jobs      = 4,
                                  objective   = 'binary:logistic',
                                  max_depth   = 10,
                                  n_estimators= 20,
                                  booster     = 'gbtree')
    boost_clf.fit (X_train[:, 0:i], Y_train)
    
    prediction = boost_clf.predict(X_test[:, 0:i])
    ROC_AUC_score = roc_auc_score          (Y_test, prediction)
    Avg_precision = average_precision_score(Y_test, prediction)
    print ("i = ", i, "ROC_AUC = ", ROC_AUC_score, "Avg_precision = ", Avg_precision)

In [ ]:
# def print_ROCinfo (model, dataLoader, device, alpha=0.5, n_dots=101):
#     model = model.to(device)
    
    
#     threshold_massive = np.linspace (0, n_dots-1, n_dots, dtype=int)
#     TP_massive = np.zeros (n_dots)
#     FP_massive = np.zeros (n_dots)
#     FN_massive = np.zeros (n_dots)
#     TN_massive = np.zeros (n_dots)
    
#     for data in dataLoader:
#         inputs = data[0].to(device)
#         labels = data[1].to(device)

#         outputs = model(inputs)
        
#         for threshold in threshold_massive:
#             prediction = (outputs[:, 1, :, :].unsqueeze(1))>(threshold/n_dots)
#             TP_massive[threshold] += torch.sum (prediction       * labels      ).float()
#             FP_massive[threshold] += torch.sum (prediction       * (1 - labels)).float()
#             FN_massive[threshold] += torch.sum ((1 - prediction) * labels      ).float()
#             TN_massive[threshold] += torch.sum ((1 - prediction) * (1 - labels)).float()
            
#     threshold_massive = threshold_massive / (n_dots-1)
#     precision_massive = TP_massive / (TP_massive + FP_massive)
#     TPR_massive       = TP_massive / (TP_massive + FN_massive)
#     FPR_massive       = FP_massive / (FP_massive + TN_massive)

#     sum_events = TP_massive[int(len(TP_massive)/2)] + FP_massive[int(len(FP_massive)/2)] + FN_massive[int(len(FN_massive)/2)] + TN_massive[int(len(TN_massive)/2)] 
#     print ('TP = ', round(TP_massive[int(len(TP_massive)/2)] / sum_events, 6), '%')
#     print ('FP = ', round(FP_massive[int(len(FP_massive)/2)] / sum_events, 6), '%')
#     print ('FN = ', round(FN_massive[int(len(FN_massive)/2)] / sum_events, 6), '%')
#     print ('TN = ', round(TN_massive[int(len(TN_massive)/2)] / sum_events, 6), '%')
    
#     if (SAVE_INFO == True):
#         print ('TP = ', round(TP_massive[int(len(TP_massive)/2)] / sum_events, 6), '%', file=INFO_FILE)
#         print ('FP = ', round(FP_massive[int(len(FP_massive)/2)] / sum_events, 6), '%', file=INFO_FILE)
#         print ('FN = ', round(FN_massive[int(len(FN_massive)/2)] / sum_events, 6), '%', file=INFO_FILE)
#         print ('TN = ', round(TN_massive[int(len(TN_massive)/2)] / sum_events, 6), '%', file=INFO_FILE)
    
#     # plot 1 precision
#     fig1 = plt.figure(figsize=(10, 6))

#     axes = fig1.add_axes([0.1, 0.1, 0.8, 0.8])

#     axes.plot(threshold_massive, precision_massive, color='green', marker='^')

#     axes.set_xlabel('threshold')
#     axes.set_ylabel('precision')

#     if (SAVE_INFO == True):
#         plt.savefig(EXPERIMENT_DIR + 'Precision_from_threshold.png', format='png', dpi=100)
#     plt.show()
    
#     # plot 2 recall
#     fig = plt.figure(figsize=(10, 6))

#     axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])

#     axes.plot(threshold_massive, TPR_massive, color='green', marker='^')

#     axes.set_xlabel('threshold')
#     axes.set_ylabel('recall')
    
#     if (SAVE_INFO == True):
#         plt.savefig(EXPERIMENT_DIR + 'Recall_from_threshold.png', format='png', dpi=100)
#     plt.show()
    
#     # plot 3 ROC-curve
#     fig = plt.figure(figsize=(8, 8))

#     axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])

#     axes.plot(FPR_massive, TPR_massive, 'orange', marker = '^')
#     axes.plot (range(2), range(2), 'grey', ls='--')

#     axes.set_xlabel('FPR')
#     axes.set_ylabel('TPR (recall)')
#     axes.set_title('ROC-curve')

#     if (SAVE_INFO == True):
#         plt.savefig(EXPERIMENT_DIR + 'ROC_curve.png', format='png', dpi=100)
#     plt.show()
    
#     del model
#     del inputs
#     del labels

In [ ]:
def print_INFO_to_file():
    if (SAVE_INFO == True):
        print ('ORIGIN_LATITUDE        =', ORIGIN_LATITUDE                       , file=INFO_FILE)
        print ('ORIGIN_LONGITUDE       =', ORIGIN_LONGITUDE                      , file=INFO_FILE)
        print ('LEFT_BORDER            =', LEFT_BORDER                           , file=INFO_FILE)
        print ('RIGHT_BORDER           =', RIGHT_BORDER                          , file=INFO_FILE)
        print ('DOWN_BORDER            =', DOWN_BORDER                           , file=INFO_FILE)
        print ('UP_BORDER              =', UP_BORDER                             , file=INFO_FILE)
        print ('N_CELLS_HOR            =', N_CELLS_HOR                           , file=INFO_FILE)
        print ('N_CELLS_VER            =', N_CELLS_VER                           , file=INFO_FILE)
        print (' '                                                               , file=INFO_FILE)
        print ('OBSERVED_DAYS          =', OBSERVED_DAYS                         , file=INFO_FILE)
        print ('DAYS_TO_PREDICT        =', DAYS_TO_PREDICT                       , file=INFO_FILE)
        print ('STEP                   =', STEP                                  , file=INFO_FILE)
        print ('TESTING_DAYS           =', TESTING_DAYS                          , file=INFO_FILE)
        print ('HEAVY_QUAKE_THRES      =', HEAVY_QUAKE_THRES                     , file=INFO_FILE)
        print ('LEARNING_RATE          =', LEARNING_RATE                         , file=INFO_FILE)
        print ('N_EPOCHS               =', N_EPOCHS                              , file=INFO_FILE)
        print ('EARTHQUAKE_WEIGHT      =', EARTHQUAKE_WEIGHT                     , file=INFO_FILE)
        print ('TRAIN_SHAPE            =', earthquakes_dataset_train.data.shape  , file=INFO_FILE)
        print ('TEST__SHAPE            =', earthquakes_dataset_test .result.shape, file=INFO_FILE)
        
#         print ('', , file=INFO_FILE)


In [ ]:
if (SAVE_INFO == True):
    INFO_FILE = open (EXPERIMENT_DIR + 'INFO.txt', 'w')
    
if (SAVE_INFO == True):
    print_INFO_to_file()
    
print_ROCinfo (earthquake_network,
               earthquakes_dataloader_test,
               DEVICE,
               ) 

if (SAVE_INFO == True):
    INFO_FILE.close()

In [ ]:
# class ConvNetwork_MSE (nn.Module):
    
#     def __init__(self):
#         super(ConvNetwork_CE, self).__init__()
        
#         self.features = nn.Sequential()
        
#         self.features.add_module('conv1', conv_block(     OBSERVED_DAYS    , int (OBSERVED_DAYS/2 ), 3))
#         self.features.add_module('conv2', conv_block(int (OBSERVED_DAYS/2 ), int (OBSERVED_DAYS/4 ), 3))
#         self.features.add_module('conv3', conv_block(int (OBSERVED_DAYS/4 ), int (OBSERVED_DAYS/8 ), 3))
#         self.features.add_module('conv4', conv_block(int (OBSERVED_DAYS/8 ), int (OBSERVED_DAYS/16), 3))
#         self.features.add_module('conv5', conv_block(int (OBSERVED_DAYS/16),                      1, 3))
        
#         # might be a good idea to add an extra full connected layer
        
#     def forward(self, x):
# #         print ('input  : ', x.shape)
#         x = self.features(x)
# #         print ('output : ', x.shape)
#         return x
        

In [ ]:
# def train_network_MSE(network, 
#                       device,
#                       dataloader_train,
#                       dataloader_test,
#                       epochs=164,
#                       learning_rate=0.1):
    
#     train_acc = []
#     test_acc  = []
#     net = network.to(device)

#     criterion = nn.MSELoss()
#     optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, weight_decay=0.0001, momentum=0.9)

    
#     for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
#         if epoch == epochs/2:
#             optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate/10, weight_decay=0.0001, momentum=0.9) 
#         elif epoch == epochs*3/4:
#             optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate/100, weight_decay=0.0001, momentum=0.9) 
        
#         net = net.train()        
#         epoch_accuracy = 0.0
#         epoch_elems = 0
#         for data in dataloader_train:
            
#             inputs = data[0].to(device)
#             labels = data[1].to(device)
# #             print ('inputs_shape = ', inputs.shape)
# #             print ('labels_shape = ', labels.shape)

#             # zero the parameter gradients
#             optimizer.zero_grad()

#             # forward + backward + optimize
#             outputs = net(inputs)
# #             print ('outputs : ', outputs.dtype)
# #             print ('labels  : ', labels.dtype)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

# #             print (loss)
#             epoch_elems    += labels.shape[0]
#             epoch_accuracy += loss.item()*labels.shape[0]

#         epoch_accuracy /= epoch_elems
#         train_acc.append(epoch_accuracy)
        
        
#         # calculating test accuracy
#         epoch_accuracy = 0.0
#         epoch_elems = 0
#         for data in dataloader_test:
#             inputs = data[0].to(device)
#             labels = data[1].to(device)   
#             outputs = net(inputs)
#             loss = criterion(outputs, labels)
            
#             epoch_elems    += labels.shape[0]
#             epoch_accuracy += loss.item()*labels.shape[0]

#         epoch_accuracy /= epoch_elems
#         test_acc.append(epoch_accuracy)
        
        
#         print('Epoch : ', epoch, 'acc_train : ', round (train_acc[-1], 4), 'acc_test : ', round (test_acc[-1], 4))

#     print('Finished Training')
    
#     plt.plot(train_acc, label='Train')
#     plt.plot(test_acc , label='Test' )
#     plt.legend()
#     plt.grid()

In [ ]:
# earthquake_netowrk = ConvNetwork_MSE ()
# train_network_MSE (earthquake_netowrk,
#                    torch.device(DEVICE),
#                    earthquakes_dataloader_train,
#                    earthquakes_dataloader_test,
#                    epochs=200,
#                    )

In [ ]:
# def find_mistake (x):
#     assert (torch.sum((x < 0.0) + (x > 1.0)) == 0)

In [ ]:
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# print ('input  : ', input.shape, input.dtype)
# print (input)
# print (torch.sum (input, dim = 0))
# print ('target : ', target.shape, target.dtype)
# print (target)
# output = loss(input, target)
# output.backward()

In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader

# class someDataset (Dataset):
#     def __init__(self):
#         self.data = torch.ones ([100, 32, 10, 10])
#         self.labels = torch.ones ([100, 1, 10, 10])
#         self.len  = self.data.shape[0]
        
#         print (self.data.shape)
#         print (self.labels.shape)
        
#     def __len__ (self):
#         return self.len
    
#     def __getitem__(self, idx):
#         print ('data_shape = ', self.data[idx].shape)
#         print ('result_shape = ', self.labels[idx].shape)
#         return self.data[idx], self.labels[idx]

In [ ]:
# some_dataset = someDataset()
# dataloader = DataLoader (some_dataset,
#                          batch_size=32,
#                          shuffle=True,
#                          num_workers=1,
#                          )

# for i, batch in enumerate(dataloader, 0):
#     data = batch[0]
#     print (i, 'data ', data.shape)
#     labels = batch[1]
#     print (i, 'labels ', labels.shape)
    


# # eartquakes_dataloader_train = DataLoader(earthquakes_dataset_train,
# #                                          batch_size=33,
# #                                          shuffle=True,
# #                                          num_workers=4)
